In [63]:
import re
import pandas as pd
from cleantext import clean

## PART 1

In [72]:
#cpr regex som er pænt ligeglad om en dato er valid eller ej (dit ejet problem)
cprRE = r"([0-2][0-9]|3[0-1])(0[0-9]|1[0-2])([0-9]{2})-?([0-9]{4})"



In [90]:
#test af cpr regex (chatgpt used to generate fake data)
cpr_list = [
    "200144-7290",
    "020164-1952",
    "1303494374",
    "2006681112",
    "280659-4979",
    "050421-8939",
    "1401704600",
    "2708256898",
    "141100-6779",
    "211183-3482",
    "2507371827",
    "2510439716",
    "1601594544",
    "0808259665",
    "2102077509",
    "2503102352",
    "130284-0998",
    "1705956426",
    "180263-5407",
    "0604882964",
]


for cpr in cpr_list:
    match = re.match(cprRE, cpr)
    if not match:
        print(cpr)

invalid_cpr_list = [
    "1234567890",      # Missing separator but invalid format (should be DDMMYYIIII)
    "123456-789",      # Too short (only 3 digits in the last part)
    "12345-67890",     # Too many digits in the first part
    "1234-567890",     # Incorrect separator placement
    "123456_7890",     # Wrong separator (underscore instead of hyphen)
    "12A456-7890",     # Contains a letter in the date part
    "123456-78B0",     # Contains a letter in the identifier part
    "000000-0000",     # All zeros (not realistic)
    "320299-1234",     # Invalid date (February 32nd does not exist)
    "290200-1234",     # Invalid leap year date (February 29th on a non-leap year)
    "311199-12345",    # Too many digits in the last part
    "311299-",         #Missing last 4 digits
    "-3112991234",     # Starts with a separator
    "311299--1234",    # Double separator
    "999999-9999",     # Unrealistic high values
    "010120 1234",     # Space instead of hyphen
    "010120123",       # Last part too short
    "abcdef-ghij",     # Only letters
    "01.01.20-1234",   # Wrong date format (dots instead of numbers)
    "310499/5678"      # Wrong separator (/ instead of -)
]



for cpr in invalid_cpr_list:
    match = re.match(cprRE, cpr)
    if match:
        print(cpr)



000000-0000
290200-1234
311199-12345


In [65]:
def yearofbirth (input_IIII, input_YY):
    #Define IIII/YY&century relationship
    table = [
        ((1, 3999), (0, 99), 1900),
        ((4000, 4999), (0, 36), 2000),
        ((4000, 4999), (37, 99), 1900),
        ((5000, 8999), (0, 57), 2000),
        ((5000, 8999), (58, 99), 1800),
        ((9000, 9999), (0, 36), 2000),
        ((9000, 9999), (37, 99), 1900),
    ]
    
    #check input and return if a matching touples in the same row exists
    for (range_IIII, range_YY, century) in table:
        if range_IIII[0] <= input_IIII <= range_IIII[1] and range_YY[0] <= input_YY <= range_YY[1]:
            return century

    #else raise error
    raise ValueError("Invalid input")



## PART 2

In [66]:
#load data
dataPath = "data/"
newsSample = pd.read_csv(dataPath + "news_sample.csv")
nsdf = pd.DataFrame(newsSample)
nsdf = nsdf.reset_index(drop=True)  # Reset index??
print(nsdf.info())   # Check column types and missing values



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        250 non-null    int64  
 1   id                250 non-null    int64  
 2   domain            250 non-null    object 
 3   type              238 non-null    object 
 4   url               250 non-null    object 
 5   content           250 non-null    object 
 6   scraped_at        250 non-null    object 
 7   inserted_at       250 non-null    object 
 8   updated_at        250 non-null    object 
 9   title             250 non-null    object 
 10  authors           170 non-null    object 
 11  keywords          0 non-null      float64
 12  meta_keywords     250 non-null    object 
 13  meta_description  54 non-null     object 
 14  tags              27 non-null     object 
 15  summary           0 non-null      float64
dtypes: float64(2), int64(2), object(12)
memory u

### Data Inspection
------------------------------------------------------
lbbablbalba

### 4. Cleaning

In [67]:
def cleantext(input):
    #make text lowercase
    input_lowercase = input.lower()
    #remove multiple white spaces, tabs, or new lines

    #numbers, dates, emails, and URLs should be replaced by "<NUM>", "<DATE>", "<EMAIL>" AND "<URL>", respectively.
    numberRE = r"[0-9]+"
    datRE = r""
    emailRE = r"^[a-zA-Z0-9._+-]+@(gmail|hotmail).(net|com|org)"
    urlRE = r""

### 5.5) Cleaning Using Library

In [68]:
def clean_text_column(text):
    if isinstance(text, str):
        return clean(text, lower=True, replace_with_url="<URL>", replace_with_email="<EMAIL>", replace_with_number="<NUMBER>")
    return text  # Return unchanged if not a string

def cleanText(data):
    data['content'] = data['content'].apply(clean_text_column)  # Apply function
    return data

In [69]:
nsdf_cleaned = cleanText(nsdf)

In [71]:
print(nsdf_cleaned['content'].head)

<bound method NDFrame.head of 0      sometimes the power of christmas will make you...
1      awakening of 12 strands of dna "reconnecting w...
2      never hike alone: a friday the 13th fan film u...
3      when a rare shark was caught, scientists were ...
4      donald trump has the unnerving ability to abil...
                             ...                        
245    prison for rahm, god's work and many others\nh...
246    4 useful items for your tiny home\nheadline: b...
247    former cia director michael hayden said thursd...
248    antonio sabato jr. says hollywood's liberal el...
249    former u.s. president bill clinton on monday c...
Name: content, Length: 250, dtype: object>
